### EXPLICACION DEL TRABAJO DE PROCESADO DEL DATA SET DE PELICULAS

#### CONSIGNA:

 Deberá adaptar los datos de la siguiente forma:
 - Descartar las películas que no tienen “overview”.
 - Descartar las películas cuyo idioma original tenga más de 2 caracteres.
 - Tomar las 100 palabras más comunes de todos los overviews combinados.
 - Para generar el ”Overview” de cada película para el archivo de salida se deberán
 descartar las palabras que se encuentren dentro de las 100 más comunes de todos los
 overviews y se deberá generar un string con 3 palabras tomadas al azar y ordenadas al
 azar (ver random.sample()) de ese resultado. Nota: luego de filtrar las palabras
 comunes algunos overviews pueden tener menos de 3 palabras, en esos casos se
 tomarán las palabras que se pueda.
 - Se utilizarán como datos de las tarjetas: “Genre”, “Original_Language”, “Vote_Average”,
 “Release_Date” y 3 palabras aleatorias del overview separadas por “;”. Como dato a
 adivinar se utilizará “Title”. Descartar el resto de las columnas.
 - El archivo resultante deberá tener las siguientes columnas (en este orden específico):
 “Genre”, “Original_Language”, “Release_Date”, “Vote_Average”, “Overview” y “Title”

#### URL: https://www.kaggle.com/datasets/disham993/9000-movies-dataset

### IMPLEMENTACION DE LA RESOLUCION:

Primero vamos a importar las librerias a utilizar que van a ser necesarias para el procesamiento de los datos

In [ ]:
import csv
import os
from collections import Counter
import random
import pandas as pd

Ahora vamos a empezar la lectura del csv buscando la ruta del directorio actual donde se encuentra el archivo donde contiene la lista de peliculas necesarias para lo primero que pide la consigna

In [ ]:
ubicacion = os.path.join("folder_csv")
carpeta_actual = os.path.join("src")
carpeta_sig = os.path.join("core")
ruta_act = os.path.join(os.getcwd(),carpeta_actual,carpeta_sig) 
ruta_final = os.path.join(ruta_act,ubicacion)

with open(os.path.join(ruta_final,'mymoviedb.csv'), "r",encoding="utf8") as logs_peliculas:
     lector = csv.reader(logs_peliculas, delimiter=",")
     header, datos = next(lector), list(lector)

Utilizamos el with ya que permite cerrar el archivo sin poner el open y utilizamos el path.join para unir la direccion del directorio actual donde se encuentra el archivo en donde se ubica. Tambien vamos a quitar del encabezado por asi decirlo para poder manejar los datos en bruto mas facil el manejo de los datos. 

Ahora vamos a juntar todas las overviews de las peliculas en una sola lista para poder generar la lista de las 100 palabras mas comunes en todas las overwievs del csv:

In [ ]:
     word_list = []
     for linea in datos:
         cadena = linea[2].split()
         for palabra in cadena: #Recorro la cadena actual de palabras
             word_list.append(palabra)

Y ahora vamos a buscar las palabras mas comunes que se encuentran en todos los overviews utilizando el iterador Counter que contiene el most_commons() que busca coincidencias 

In [ ]:
     word_list = list(dict(Counter(word_list).most_common(100)).keys()) 

Como ya tenemos la lista de las 100 palabras que mas aparecen en los overviews,ahora vamos a realizar un proceso para generar 3 palabras al azar unidas por ; que van a ser provistas al nuevos csv cuando se hayan creado

In [ ]:
     def generar_cadena(cadena):
         """Busca palabra por palabra en la cadena de la linea de overviews y va descartando del la 
         linea si la palabra actual esta en la word list que contiene las 100 palabras que aparecen 
         en los overviews combinados"""
         data = []
         for palabra,word in zip(cadena,word_list):
             if palabra != word:
                 data.append(palabra)
         data = ";".join(random.sample(data,min(len(data),3)))
         return data

Data es la lista que son generadas a partir del descarte de palabras que se encuentran en la lista combinada y que tambien estan en la overview actual. A lo ultimo utilizamos el sample por que devuelve una muestra de 3 palabras. Se utilizo el min(tamaño de la lista actual,3) sobre sample por que genera excepciones ValueError y esta solucion evita que suceda esos errores ya que podia haber muchas muestras negativas. 

Ahora vamos a realizar la creacion del csv utilizando un diccionario y DictWriter para escribir sobre el. Primero realizamos la obtencion de la lista completa con los paises cuyo lenguaje consiste en 2 caracteres junto con descartar la cantidad que haya de peliculas sin overviews y luego vamos a usar la cadena actual para generar la lista de 3 palabras al azar con generar_cadena()

In [ ]:
     refined_data_list = []
     for linea in datos:
         idioma = len(linea[6])
         cadena = linea[2].strip(",").split() #Lista de palabras de la cadena actual del overview
         if idioma == 2 and len(linea[2]) > 0:
             refined_data_list.append({'Genre': linea[7], 'Original_Language': linea[6],'Release_Date': linea[0],'Vote_Average': linea[5],'Overview': generar_cadena(cadena),'Title': linea[1]})

refined_data_list es donde se van a almacenar todos los datos resultantes que van a ir al csv y como es una lista de diccionarios va a simplificar la escritura del csv ya que recorro menos el archivo. Ahora vamos a generar el csv donde estan los resultados llamado  **peliculas_figurace** 

Vamos a utilizar pandas para generar los datos.
Ahora establecemos el nombre de los campos con la lista llamada field_names y escribimos su encabezado y para finalizar escribirmos el encabezado que seria las palabras en field_names y la lista que se genero anteriormente conocida como refined_data_list. 

In [ ]:
field_names =['Genre','Original_Language','Release_Date','Vote_Average','Overview','Title']
df = pd.DataFrame(refined_data_list,columns=field_names)
df.to_csv(os.path.join(os.getcwd(),ruta_final,'peliculas_figurace.csv'), index=False) 